In [35]:
import datetime as dt
import sys
sys.path.insert(0,'../..')
from typing import List
import numpy as np
import matplotlib.pyplot as plt
import json
import os
import numpy as np
from scipy.special import comb
import pandas as pd
import seaborn
import pprint
import deepdiff
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

from rivapy.tools.datetime_grid import DateTimeGrid
from rivapy.models.residual_demand_fwd_model import WindPowerForecastModel, MultiRegionWindForecastModel, LinearDemandForwardModel
from rivapy.instruments.ppa_specification import GreenPPASpecification
from rivapy.models.residual_demand_model import SmoothstepSupplyCurve
from rivapy.models import OrnsteinUhlenbeck
from rivapy.pricing.green_ppa_pricing import GreenPPADeepHedgingPricer, DeepHedgeModel


import analysis

from IPython.display import display, HTML
#display(HTML("<style>.container { width:80% !important; }</style>"))

%load_ext autoreload
%autoreload 2
%matplotlib inline




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [61]:
wind_onshore = WindPowerForecastModel(region='Onshore', speed_of_mean_reversion=2.0, volatility=4.0)
wind_offshore = WindPowerForecastModel(region='Offshore', speed_of_mean_reversion=2.0, volatility=3.0)
regions = [ MultiRegionWindForecastModel.Region( 
                                    wind_onshore,
                                    capacity=1000.0,
                                    rnd_weights=[0.8,0.2]
                                ),
           MultiRegionWindForecastModel.Region( 
                                    wind_offshore,
                                    capacity=100.0,
                                    rnd_weights=[0.2,0.8]
                                )
           
          ]
wind = MultiRegionWindForecastModel('Wind_Germany', regions)

model = LinearDemandForwardModel(wind_power_forecast=wind, 
                                 x_volatility = 4.0 , 
                                 x_mean_reversion_speed = 0.1,
                                 power_name= 'Power_Germany')

In [71]:
val_date = dt.datetime(2023,1,1)
strike = 1.0 #0.22
transaction_cost = 0.01
spec = GreenPPASpecification(udl='Power_Germany',
                            technology = 'Wind',
                             location = 'Onshore',
                             schedule = [val_date + dt.timedelta(days=2)], 
                             fixed_price=strike,
                             transaction_cost=transaction_cost,
                             max_capacity = 1.0)

In [72]:
repo = analysis.Repo('./experiments/')
repo.results.keys()

dict_keys(['a7b4461a3353337f96e58188db2087eb4cd90335', '75b5248ddc4b09a2e0e1194910fdc05bce4e3112', '91c071ac0f3214cd0e65e3e636755f80b8f8a5e7', 'c1ab45b4ad248d3cd152c78631088cc7f1c61ce4', '7ef18c826acc39073b98f0e630cc1861ee7504b0', '4a0107c15c4488cab77dec3325fcadc814a96384'])

In [73]:
pricing_results = repo.run(val_date, 
                           spec, model, 
                           initial_forecasts={'Onshore': [0.8],
                                        'Offshore': [0.6]},
                           power_fwd_prices=[1.0],
                           forecast_hours=[10, 14, 18],
                           additional_states=['Offshore'],
                           depth=3, 
                           nb_neurons=64, 
                           n_sims=100_000, 
                           regularization=0.0,
                           epochs=1, verbose=1,
                           tensorboard_logdir = 'logs/' + dt.datetime.now().strftime("%Y%m%dT%H%M%S"), 
                           initial_lr=5e-4,
                           decay_steps=8_000,
                           batch_size=100, decay_rate=0.8, seed=42)

3125/3125 [==============================] - 22s 5ms/step - loss: 1.7398e-05


INFO:tensorflow:Assets written to: ./experiments//d87b2c0fa3e81977f043dc1577f26386b2021d19//delta_model\assets


INFO:tensorflow:Assets written to: ./experiments//d87b2c0fa3e81977f043dc1577f26386b2021d19//delta_model\assets
